In [1]:
import pandas as pd
from neo4j import GraphDatabase
import datetime
from datetime import timedelta
pd.set_option('display.max_colwidth', None)

In [2]:
#Establish connection to Neo4j
df_pw = pd.read_csv("pw.csv", delimiter=",")
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [3]:
#Load wallets of vendors known to be active on both Berlusconi & Cryptonia Market as well as all positive BM reviews.
vendor_wallets = pd.read_csv('vendor_wallets_filtered2.csv')
berlusconi_reviews  = pd.read_csv('reviews_price_btc.csv')
berlusconi_reviews.time = pd.to_datetime(berlusconi_reviews.time)

In [4]:
#Query for TAs matching the Berlusconi Market pattern and pointing to an address from the vendor wallets.
start_transactions = []

for i in vendor_wallets.itertuples():

    address = i.Address
    observedChange = str(0.01 * 100000000)

    neo_query = session.run("""
    WITH 
        '""" + address + """' as start,
        """ + observedChange + """ as usualChange
    MATCH (payout:Transaction)-[RECEIVES]->(vendor:Address)
    WHERE vendor.address = start and payout.outDegree = 2
    WITH payout, usualChange
    MATCH (payout)-[r1:RECEIVES]->(change1:Address)-[s:SENDS]->(payout2:Transaction)-[r2:RECEIVES]->(change2:Address)
    WHERE 0.85 * usualChange <= r1.value <= 1.15 * usualChange
    AND 0.85 * usualChange <= r2.value <= 1.15 * usualChange
    AND r1.value = s.value
    AND (change1.address STARTS WITH "3" OR change1.address STARTS WITH "bc1")
    AND (change2.address STARTS WITH "3" OR change2.address STARTS WITH "bc1")
    AND payout2.outDegree = 2
    RETURN payout.txid
    """)
    
    for r in neo_query:
        start_transactions.append(str(r[0]))

    
display(start_transactions)

['e95f53d3eead4f89ec569dd37638030d8515eefed8593a5330aa0b6d7ac39a45',
 'caa10bfd483632ba44aa6c860e2f30cab581341b21c819d316b99f0e277bb4f6',
 '57e34d74f1cd5528253c28ccd938c2a9307545b9a2263cecb29cc75941bbe02a',
 '0b9d70cf1e2a70e0f33b4620deb5f6afde7c3547774a3d4ee5e973aeb873a31e',
 '433e1b460e8d67e5fa55aa780154cc27080b5635132c179887795b419b80010b',
 '972c2b76db5127b11421ea4508ca5d07f044f4e12eaf4123b40e8cb1bb15f719',
 'd3b1ddda7c8f57109d77cb5817e894f87f2292ee58c30b423b6411dff911e90e',
 'b864b9953a392264793c4ef9cd9b2075154d6777492a6b755227ff3209fb9083',
 '689a4b8511c88a9dcd5f766dd42d1d2f1524aeb80c57eb1a169c874549592bd9',
 'b76e50c92da657c31b2fdb30eced4013f0f7ee997901d161da4bb08cec4da424',
 '272ee0bad42e4c63d8625ca1bcf6e233d4233ffba28a958bf93a1db3e2ebe20d',
 'f05773d24f31ac444b6da4fa117ab0ebc44e01e2f5081360c46794b997d5c74c',
 '063a367c377d4df9041ceb9702d20c7969cdbdeabab4f195d0c24b714f668a16',
 '51f8ee15f6746e67df3b333d45bcda27e3028d5bdc6358d1c4366de28fa43c1e',
 '5a4e70abffa7e043e349921546ab686b

In [5]:
#Combine starting points from the vendor wallets with previously identified BM payout transaction using manual inspection.
payout_TAs = ["0debe14ba12bc429ac4933dc61aa64695ee1e7e41ed9e9477e2e4382527a063b", 
              "e8748fe252cfafaf7295b31236c7fb6bff899b8b4a70bdaa781bbd62651507e6", 
              "4993349442303968c94b4a0b05ff49fc8543bcfd1c677bccc6001260891ca58d", 
              "c54c234e4e37feb1447f1a41ca6fd7293ce3af2ca3dd53c99a24007534b810fd", 
              "7b8d6e19864889cb639efe748f7d05840ace40f8353ca77660d96a04132ace1b", 
              "0d5da4185c73ffba8bc0407433761d61d6574d8499dea7a9f0b9268ac49b4357",
              "e8babd3eefe845c02af052c3564e1e0a101df4d04f13f65445b9558eafced6d9",
              "35f3515130514b209691f3b5478809f3488cb02ad5325ac262bae4053dd408f9",
              "aced23e9637b1983f31558e14b6068a26b0b0c53d6154847e8506b5dd01e0e89",
              "0c4d1db76001531703ac1d0c31d8950dcc57faeb3e2cff82cb70171105f57a4b",
              "00dee16249a3cce46cef9d2b0b07cb789cac214e5e5266fbf082e5ed43a36d75"]
payout_TAs = payout_TAs + start_transactions
len(payout_TAs)

274

In [ ]:
#DF for final results
potential_vendors = pd.DataFrame()
observedChange = str(0.01 * 100000000)
not_covered = payout_TAs
covered = []
ctr = 0

while not_covered:
    ctr = ctr+1
    neo_results = []
    neo_results2 = []
    neo_results3 = []
    neo_results4 = []
    
    # Pop the next txid to use
    txid = not_covered.pop()
    covered.append(txid)
    
    #Follow the change chain forward
    #Matches the txid and returns the identified vendor, the amount paid to him,
    #the date of the payout and the next payout TA in the chain
    neo_query = session.run("""
    WITH '""" + txid + """' as start,
        """ + observedChange + """ as usualChange
    MATCH (payout:Transaction)-[r:RECEIVES]->(change:Address)
    WHERE payout.txid = start AND 0.85 * usualChange <= r.value <= usualChange * 1.15 AND payout.outDegree = 2
    WITH payout, change
    MATCH (payout)-[r:RECEIVES]->(vendor:Address)
    WHERE vendor <> change 
    WITH vendor.address as vendor_address, r.value as amount, payout.date as date, change
    MATCH (change)-[SENDS]->(next_payout:Transaction)
    RETURN DISTINCT vendor_address, amount, date, next_payout.txid
    """)
    
    #Resultset should be empty if the end of the payout chain is reached
    if neo_query:
        for r in neo_query:
            neo_results.append(r[0])
            neo_results2.append(r[1])
            neo_results3.append(r[2])
            neo_results4.append(r[3])

        df_neo4j = pd.DataFrame()
        #The line below can be uncommented to view where the change chain ends going forward.
        #df_neo4j["Payout TA"] = pd.Series(txid)
        df_neo4j["VendorAddress"] = pd.Series(neo_results)
        df_neo4j["Amount"] = pd.Series(neo_results2)
        df_neo4j["Date"] = pd.Series(neo_results3)
        potential_vendors = pd.concat([potential_vendors,df_neo4j])
        
        #check for duplicates to reduce run time
        if (neo_results4):
            if str(neo_results4[0]) not in covered and str(neo_results4[0]) not in not_covered:
                not_covered.append(str(neo_results4[0]))

    #Follow the change chain backwards
    vendor_addresses = []
    amounts = []
    dates = []
    previous_payouts = []

    neo_query = session.run("""
    WITH '""" + txid + """' as start,
        """ + observedChange + """ as usualChange
    MATCH (change:Address)-[s:SENDS]->(payout2:Transaction)
    WHERE payout2.txid = start AND 0.85 * usualChange <= s.value <= usualChange * 1.15 AND payout2.outDegree = 2
    WITH change, s
    MATCH (payout1:Transaction)-[r:RECEIVES]->(change)
    WHERE r.value = s.value
    WITH payout1, change
    MATCH (payout1)-[r:RECEIVES]->(vendor:Address)
    WHERE change <> vendor AND payout1.outDegree = 2
    RETURN DISTINCT vendor.address, r.value, payout1.date, payout1.txid
    """)

    for r in neo_query:
        vendor_addresses.append(r[0])
        amounts.append(r[1])
        dates.append(r[2])
        previous_payouts.append(r[3])

    df_neo4j = pd.DataFrame()
    #The line below can be uncommented to view where the change chain ends going backwards.
    #df_neo4j["Payout TA"] = pd.Series(txid)
    df_neo4j["VendorAddress"] = pd.Series(vendor_addresses)
    df_neo4j["Amount"] = pd.Series(amounts)
    df_neo4j["Date"] = pd.Series(dates)
    potential_vendors = pd.concat([potential_vendors,df_neo4j])
    #Drop duplicates so the df doesn't bloat up 
    potential_vendors = potential_vendors.drop_duplicates()

    #check for duplicates to reduce run time
    for i in previous_payouts:
        if i not in covered and i not in not_covered:
            not_covered.append(i)
            
    if (ctr % 100 == 0):
        potential_vendors.to_csv("potential_vendors.csv")
session.close()
potential_vendors = potential_vendors.sort_values(by="Date",ascending=True)
potential_vendors = potential_vendors.reset_index(drop=True)
display(potential_vendors)
potential_vendors.to_csv("potential_vendors.csv")

/tmp/ipykernel_822767/3048974890.py:83: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["VendorAddress"] = pd.Series(vendor_addresses)
/tmp/ipykernel_822767/3048974890.py:84: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["Amount"] = pd.Series(amounts)
/tmp/ipykernel_822767/3048974890.py:85: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["Date"] = pd.Series(dates)
/tmp/ipykernel_822767/3048974890.py:45: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j["VendorAddress"] = pd.Serie

In [ ]:
print(len(covered))

#For every starting point: Follows the change chain forward and collects all potential vendor addresses + payout amount & date

#DF for final results
potential_vendors = pd.DataFrame()
observedChange = str(0.01 * 100000000)
payouts = payout_TAs.copy()

while payouts:
    #print(payout_TAs)
    neo_results = []
    neo_results2 = []
    neo_results3 = []
    neo_results4 = []
    
    # Pop the next txid to use
    txid = payouts.pop()
    
    # Matches the txid and returns the identified vendor, the amount paid to him,
    # the date of the payout and the next payout TA in the chain
    neo_query = session.run("""
    WITH '""" + txid + """' as start,
        """ + observedChange + """ as usualChange
    MATCH (payout:Transaction)-[r:RECEIVES]->(change:Address)
    WHERE payout.txid = start AND 0.85 * usualChange <= r.value <= usualChange * 1.15 AND payout.outDegree = 2
    WITH payout, change
    MATCH (payout)-[r:RECEIVES]->(vendor:Address)
    WHERE vendor <> change 
    WITH vendor.address as vendor_address, r.value as amount, payout.date as date, change
    MATCH (change)-[SENDS]->(next_payout:Transaction)
    RETURN DISTINCT vendor_address, amount, date, next_payout.txid
    """)
    
    #Resultset should be empty if the end of the payout chain is reached
    if neo_query:
        for r in neo_query:
            neo_results.append(r[0])
            neo_results2.append(r[1])
            neo_results3.append(r[2])
            neo_results4.append(r[3])

        df_neo4j = pd.DataFrame()
        #The line below can be uncommented to view where the change chain ends going forward.
        #df_neo4j["Payout TA"] = pd.Series(txid)
        df_neo4j["VendorAddress"] = pd.Series(neo_results)
        df_neo4j["Amount"] = pd.Series(neo_results2)
        df_neo4j["Date"] = pd.Series(neo_results3)
        potential_vendors = pd.concat([potential_vendors,df_neo4j])
        
        if neo_results4:
            payouts.append(str(neo_results4[0]))

#Follows the change chain backwards and collects all potential vendor addresses + payout amount and date
payouts = payout_TAs.copy()

while payouts:
    vendor_addresses = []
    amounts = []
    dates = []
    previous_payouts = []
    
    txid = payouts.pop()

    neo_query = session.run("""
    WITH '""" + txid + """' as start,
        """ + observedChange + """ as usualChange
    MATCH (change:Address)-[s:SENDS]->(payout2:Transaction)
    WHERE payout2.txid = start AND 0.85 * usualChange <= s.value <= usualChange * 1.15 AND payout2.outDegree = 2
    WITH change, s
    MATCH (payout1:Transaction)-[r:RECEIVES]->(change)
    WHERE r.value = s.value
    WITH payout1, change
    MATCH (payout1)-[r:RECEIVES]->(vendor:Address)
    WHERE change <> vendor AND payout1.outDegree = 2
    RETURN DISTINCT vendor.address, r.value, payout1.date, payout1.txid
    """)

    for r in neo_query:
        vendor_addresses.append(r[0])
        amounts.append(r[1])
        dates.append(r[2])
        previous_payouts.append(r[3])

    df_neo4j = pd.DataFrame()
    #The line below can be uncommented to view where the change chain ends going backwards.
    #df_neo4j["Payout TA"] = pd.Series(txid)
    df_neo4j["VendorAddress"] = pd.Series(vendor_addresses)
    df_neo4j["Amount"] = pd.Series(amounts)
    df_neo4j["Date"] = pd.Series(dates)
    potential_vendors = pd.concat([potential_vendors,df_neo4j])

    for i in previous_payouts:
        payouts.append(i)
        
session.close()
potential_vendors = potential_vendors.sort_values(by="Date",ascending=True)
potential_vendors = potential_vendors.reset_index(drop=True)
display(potential_vendors)

In [ ]:
#Drop duplicates in case some of the change chains overlapped.
potential_vendors = potential_vendors.drop_duplicates()
display(potential_vendors)

In [ ]:
berlusconi_reviews

In [ ]:
#For every potential vendor address check whether a matching BM review exists. Gather matches in payout_review_matches.
payout_review_matches = pd.DataFrame()

for idx, payout in potential_vendors.iterrows():
    for idx2, review in berlusconi_reviews.iterrows():
        if (payout.Date == pd.to_datetime(review.time).date() + timedelta(days=2)) & (0.9 * review.price_btc_reviews_satoshi <= payout.Amount <= review.price_btc_reviews_satoshi):
            temp = pd.DataFrame({"VendorAddress": [payout.VendorAddress],"Date": [payout.Date],"PayoutAmount": [payout.Amount],"ReviewAmount": [review.price_btc_reviews_satoshi],"amountDifference": [abs(review.price_btc_reviews_satoshi-payout.Amount)],"VendorID": [review.vendor_id],"ReviewID": [review.vendor_review_id]})
            payout_review_matches = pd.concat([payout_review_matches, temp])
            

payout_review_matches = payout_review_matches.reset_index(drop=True)
display(payout_review_matches.sort_values(by=["VendorAddress", "ReviewAmount"], ascending=[True, True]))                                                     

In [ ]:
payout_review_matches.to_csv("payout_review_matches2.csv")